In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

final_list = []
for i in range(1, 48000,200):
    try:
        page = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset=' + str(i)
        resp = requests.get(page)
        soup = BeautifulSoup(resp.text, 'lxml')
        table_data = [x.text for x in soup.select('tr td')[0:1600]]  # trial and error to get the exact positions
        temp_list = [table_data[i:i+8] for i in range(0, len(table_data[:-7]), 8)] # put every 5 values in a row
        for temp in temp_list:
            final_list.append(temp)
        if not i%10:
            print('getting page:', i)
    except Exception as e:
        break
print('scraped pages:', i)
        
na_year_idx = [i for i, x in enumerate(final_list) if x[3] == 'n/a']  # get the indexes of the 'n/a' values
new_years = [1998, 1999, 1960, 1973]  # got them by checking online

print(*[(i, x) for i, x in enumerate(final_list) if i in na_year_idx], sep='\n')
print('new year values:', new_years)

for na_year, new_year in zip(na_year_idx, new_years):
    final_list[na_year][3] = new_year
    print(final_list[na_year], new_year)
    

regex = '|'.join(['\$', ',', '\^'])

columns = ['rank', 'title', 'Worldwide_lifetime_Gross','Domestic_Lifetime_Gross','Domestic_%','Foreign_Lifetime_Gross', 'Foreign_%','year']

boxoffice_df = pd.DataFrame({
    'rank': [(x[0]) for x in final_list],  # convert ranks to integers
    'title': [x[1] for x in final_list],  # get titles as is
    'Worldwide_Lifetime_Gross': [int(re.sub(regex, '', x[2])) for x in final_list],  # remove special characters and convert to integer
    'Domestic_Lifetime_Gross':[(re.sub(regex, '', x[3])) for x in final_list],
    'Domestic_%':[(re.sub(regex, '', x[4])) for x in final_list],
    'Foreign_Lifetime_Gross':[int(re.sub(regex, '', x[5])) for x in final_list],
    'Foreign_%':[(re.sub(regex, '', x[6])) for x in final_list],
    'year': [int(re.sub(regex, '',str(x[7]))) for x in final_list],  # remove special characters and convert to integer
})
print('rows:', boxoffice_df.shape[0])
print('columns:', boxoffice_df.shape[1])
print('\ndata types:')
print(boxoffice_df.dtypes)
boxoffice_df.to_csv('../desktop/file.csv', index=False)

scraped pages: 47801

new year values: [1998, 1999, 1960, 1973]
rows: 48000
columns: 8

data types:
rank                        object
title                       object
Worldwide_Lifetime_Gross     int64
Domestic_Lifetime_Gross     object
Domestic_%                  object
Foreign_Lifetime_Gross       int64
Foreign_%                   object
year                         int64
dtype: object
